In [37]:
#5
import pywt
import os
from scipy.fft import dct, idct
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import convolve2d
from math import sqrt
import sys
sys.path.append(os.path.join(os.getcwd(), 'common'))
import detection
import utility
import attacks
from tqdm import tqdm
import embedding
import importlib
importlib.reload(utility)
importlib.reload(detection)
importlib.reload(attacks)
importlib.reload(embedding)


<module 'embedding' from '/home/make/multimedia/multimedia-data-security/common/embedding.py'>

In [38]:
def awgn(img, std, seed):
  mean = 0.0   # some constant
  np.random.seed(seed)
  attacked = img + np.random.normal(mean, std, img.shape)
  attacked = np.clip(attacked, 0, 255)
  return np.uint8(attacked)

def wpsnr(img1, img2):
  img1 = np.float32(img1)/255.0
  img2 = np.float32(img2)/255.0
  difference = img1-img2
  same = not np.any(difference)
  if same is True:
      return 9999999
  w = np.genfromtxt('csf.csv', delimiter=',')
  ew = convolve2d(difference, np.rot90(w,2), mode='valid')
  decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
  return decibels





img_path='lena_grey.bmp'
N = 1024

#v = 'additive'
#alpha = 24 
v = 'multiplicative'
alpha = 0.5
np.random.seed(seed=123)


image = cv2.imread(img_path, 0)
mark = np.load('ammhackati.npy')
watermarked1 =  embedding.embedding(image,mark,  alpha)




attack_functions = attacks.attack_incremental_paramters
progress_bar = tqdm(attack_functions, desc="Applying attacks")
history = []
succesfull_attack = []
for attack_fn in progress_bar:
     param = attack_fn['start']
     detected = 1
     attack_name = ''
     while param <= attack_fn['end']:
        attacked,attack_name,usd = attack_fn['function'](watermarked1, param)
        detected = detection.detection(image, watermarked1,attacked, alpha, 2)
        wpsnr_attacked = wpsnr(watermarked1, attacked)
        progress_bar.set_postfix({"attack":attack_name , "wpsnr": wpsnr_attacked,"detected":detected,"param":usd})
        #utility.visualize_images_with_desc([image, watermarked1, attacked], ['Original', 'Watermarked', 'Attacked'])
        param += attack_fn['increment_params']

        history.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd})
        if detected == 0:
            succesfull_attack.append({"attack":attack_name , "wpsnr": wpsnr_attacked,"param":usd})
            break

     
for h in succesfull_attack:
    print(h)
print("History")
for h in history:
    print(h)


print('--------------------------')
print('\n')


Applying attacks: 100%|██████████| 8/8 [00:48<00:00,  6.05s/it, attack=DWT_MEIDAN, wpsnr=47.3, detected=1, param=kernel_size: [11, 11]] 

{'attack': 'AWGN', 'wpsnr': 32.54743571145596, 'param': 'std: 35'}
{'attack': 'Blur', 'wpsnr': 43.65700347219099, 'param': 'sigma: 1'}
{'attack': 'Median', 'wpsnr': 40.76296799456945, 'param': 'kernel_size: [5, 5]'}
{'attack': 'Resizing', 'wpsnr': 35.85483307672797, 'param': 'scale: 0.3'}
History
{'attack': 'AWGN', 'wpsnr': 37.30173668435205, 'param': 'std: 20'}
{'attack': 'AWGN', 'wpsnr': 35.358884367399696, 'param': 'std: 25'}
{'attack': 'AWGN', 'wpsnr': 33.85420095734941, 'param': 'std: 30'}
{'attack': 'AWGN', 'wpsnr': 32.54743571145596, 'param': 'std: 35'}
{'attack': 'Blur', 'wpsnr': 9999999, 'param': 'sigma: 0'}
{'attack': 'Blur', 'wpsnr': 43.65700347219099, 'param': 'sigma: 1'}
{'attack': 'Sharpening', 'wpsnr': 9999999, 'param': 'sigma: 0 alpha: 0'}
{'attack': 'Sharpening', 'wpsnr': 15.270940207417613, 'param': 'sigma: 5 alpha: 5'}
{'attack': 'Sharpening', 'wpsnr': 6.366327533633082, 'param': 'sigma: 10 alpha: 10'}
{'attack': 'Sharpening', 'wpsnr': 1.429218898675516, 'param': 'si